In [15]:
import shutil, os

arch_dir = '\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\EAP627\\ParoquiadeNossaSenhoradosMilagres'
for folder, sub_folders, files in os.walk(arch_dir):
    if files:        
        vol_num = folder[folder.rfind('_') + 1:]
        ingest_dir = '\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\EAP627_ingest\\PNSM\\' + str(vol_num)
        os.makedirs(ingest_dir)
        for file in files:
            extension = file[file.rfind('.') + 1:]
            prefix = file[0:3]
            if (extension == 'jpg') and (prefix == 'IMG'):
                new_file_name = file[4:]
                shutil.move(folder + '\\' + file,ingest_dir + '\\' + new_file_name)
                

In [14]:
import errno, os, stat, shutil

def handleRemoveReadonly(func, path, exc):
  excvalue = exc[1]
  if func in (os.rmdir, os.remove) and excvalue.errno == errno.EACCES:
      os.chmod(path, stat.S_IRWXU| stat.S_IRWXG| stat.S_IRWXO) # 0777
      func(path)
  else:
      raise

eap_dir = '\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\EAP627_ingest'
to_delete = []
folders = 0

for folder, sub_folders, files in os.walk(eap_dir):
    if (folder[len(folder) - 1] == 'G') or (folder[len(folder) - 1] == 'g'):
        to_delete.append(folder)
        folders += 1
        
print(len(to_delete))
print(to_delete[0])
print(folders)

for evil_folder in to_delete:
    if os.path.exists(evil_folder):        
        shutil.rmtree(evil_folder, ignore_errors=False, onerror=handleRemoveReadonly)

97
\\libshare.library.vanderbilt.edu\vulibrary\Landers\new_organization\EAP627_ingest\AHP\0110\JPEG
97


In [8]:
os.chdir('C:\\Users\\genkindn')

In [24]:
#NSM metadata generation

master_xml_directory = input('Enter absolute directory of xml master: ')
master_xml = open(master_xml_directory,"r",encoding='utf-8')

titles = []
abstracts = []
dates = []
formats = []
conditions = []
languages = []
sources = []
folders = []
files = []
extents = []

project = "British Library EAP627"
publisher = "Slave Societies Digital Archive"
director = "Courtney J. Campbell(project director)"
metadata ="Daniel Genkins(metadata creation)"
rights = "Vanderbilt University makes these historical records available for the limited purposes of individual private study, scholarship, research, teaching, and other educational or instructional uses. No rights of any kind are granted to users with respect to photographs or graphic images published on this site."
country = "Brazil"
continent = "South America"
state = "Paraíba"
city = "São João do Cariri"
coordinates = "-7.39077,-36.53513"
types = ["Text","Ecclesiastical records"]
eras = ["Early Modern","Modern"]
subjects = ["Baptisms--Marriages--Deaths","Baptisms","Confirmations","Marriages","Deaths","Finances","Kinship"]

for line in master_xml:
    line_content = line[line.find(">") + 1: line.rfind("<")]   
    
    if "<title>" in line:
        titles.append(line_content)
    elif "<description>" in line:
        abstracts.append(line_content)
    elif "<date>" in line:
        dates.append(line_content)
    elif "<date/>" in line:
        dates.append('-1')
    elif "<format>" in line:
        formats.append(line_content)
    elif "<condition>" in line:
        conditions.append(line_content)
    elif "<language>" in line:
        languages.append(line_content)
    elif "<source>" in line:
        sources.append(line_content)
    elif "<folder>" in line:
        folders.append(line_content)
    elif "<file>" in line:
        files.append(line_content)
    elif "<extent>" in line:
        extents.append(line_content)
        
current_volume = 0

while current_volume <= 53:
    current_folder = "0" * (4 - len(str(current_volume + 1))) + str(current_volume + 1)
    print("Now working on " + current_folder)
    volume_directory = "\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\EAP627_ingest\\NSM\\" + current_folder
    current_dc = open(volume_directory + "\\DC.xml","w",encoding = "utf-8")
    current_dc.write("<oai_dc:dc xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:oai_dc=\"http://www.openarchives.org/OAI/2.0/oai_dc/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd\">")
    current_dc.write("<title>" + titles[current_volume] + "</title>")
    current_dc.write("<creator>" + project + "</creator>")
    has_date = True
    if (0 <= current_volume <= 7) or (30 <= current_volume <= 32) or (current_volume == 39) or (45 <= current_volume <= 47):
        era = eras[0]        
    elif (50 <= current_volume <= 53):
        has_date = False
    else:
        era = eras[1]
    if (current_volume == 0):
        subject = subjects[0]
    elif (1 <= current_volume <= 29):
        subject = subjects[1]
    elif (current_volume == 30):
        subject = subjects[2]
    elif (31 <= current_volume <= 38):
        subject = subjects[3]
    elif (39 <= current_volume <= 44):
        subject = subjects[4]
    elif (45 <= current_volume <= 48):
        subject = subjects[5]
    elif (49 <= current_volume <= 53):
        subject = subjects[6]
    if has_date:
        current_dc.write("<subject>" + subject + '--' + country + '--' + era + "</subject>")
    else:
        current_dc.write("<subject>" + subject + '--' + country + "</subject>")
    description = abstracts[current_volume] + " " + conditions[current_volume]
    current_dc.write("<description>" + description + "</description>")
    current_dc.write("<publisher>" + publisher + "</publisher>")
    current_dc.write("<contributor>" + director + "</contributor>")
    current_dc.write("<contributor>" + metadata + "</contributor>")
    current_dc.write("<source>" + sources[current_volume] + "</source>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
        current_dc.write("<date>" + date + "</date>")
    for typ in types:
        current_dc.write("<type>" + typ + "</type>")
    current_dc.write("<format>" + formats[current_volume] + "</format>")
    current_dc.write("<format>" + extents[current_volume] + "</format>")
    if (languages[current_volume] == "Portuguese"):
        current_dc.write("<language>por</language>")
    else:
        print("Bad language: " + str(current_volume))
    current_dc.write("<coverage>" + country + "</coverage>")
    current_dc.write("<coverage>" + continent + "--" + country + "--" + state + "--" + city + "</coverage>")
    current_dc.write("<coverage>" + coordinates + "</coverage>")
    current_dc.write("<coverage>" + era + "</coverage>")
    current_dc.write("<rights>" + rights + "</rights>")        
    current_dc.write("</oai_dc:dc>")
    current_dc.close()
    
    current_volume += 1

Enter absolute directory of xml master: C:\Users\genkindn\BL Metadata\EAP627\627_nsm_master.xml
Now working on 0001
Now working on 0002
Now working on 0003
Now working on 0004
Now working on 0005
Now working on 0006
Now working on 0007
Now working on 0008
Now working on 0009
Now working on 0010
Now working on 0011
Now working on 0012
Now working on 0013
Now working on 0014
Now working on 0015
Now working on 0016
Now working on 0017
Now working on 0018
Now working on 0019
Now working on 0020
Now working on 0021
Now working on 0022
Now working on 0023
Now working on 0024
Now working on 0025
Now working on 0026
Now working on 0027
Now working on 0028
Now working on 0029
Now working on 0030
Now working on 0031
Now working on 0032
Now working on 0033
Now working on 0034
Now working on 0035
Now working on 0036
Now working on 0037
Now working on 0038
Now working on 0039
Now working on 0040
Now working on 0041
Now working on 0042
Now working on 0043
Now working on 0044
Now working on 0045
Now 

In [26]:
current_volume = 0

while current_volume <= 53:
    current_folder = "0" * (4 - len(str(current_volume + 1))) + str(current_volume + 1)
    print("Now working on " + current_folder)
    volume_directory = "\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\EAP627_ingest\\NSM\\" + current_folder
    mods = open(volume_directory + "\\MODS.xml","w",encoding = "utf-8")
    mods.write("<mods xmlns=\"http://www.loc.gov/mods/v3\" xmlns:mods=\"http://www.loc.gov/mods/v3\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">")
    mods.write("<titleInfo>")
    mods.write("<title>" + titles[current_volume] + "</title>")
    mods.write("<subTitle/></titleInfo>")
    mods.write("<name type=\"personal\"><namePart>Courtney J. Campbell</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Project Director</roleTerm></role></name>")
    mods.write("<name type=\"personal\"><namePart>Daniel Genkins</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Metadata Creation</roleTerm></role></name>")
    mods.write("<typeOfResource>text</typeOfResource>")
    mods.write("<genre authority=\"marcgt\" />")
    mods.write("<tableOfContents/>")
    mods.write("<originInfo><dateIssued></dateIssued><copyrightDate/><issuance>monographic</issuance><edition/><publisher/><place/></originInfo>")
    mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">por</languageTerm></language>")
    mods.write("<abstract>" + abstracts[current_volume] + "</abstract>")
    mods.write("<identifier type=\"isbn\"/>")
    mods.write("<physicalDescription><form>" + conditions[current_volume] + "</form><extent>" + extents[current_volume]+ "</extent></physicalDescription>")
    mods.write("<note/>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
    mods.write("<subject><topic/><geographic>" + sources[current_volume] + "," + city + "," + state + "," + country + "</geographic><temporal>" + date + "</temporal>")
    mods.write("<hierarchicalGeographic><continent>" + continent + "</continent><country>" + country + "</country><province>" + state + "</province><region/><county/><city>" + city + "</city><citySection/></hierarchicalGeographic>")
    mods.write("<cartographics><coordinates>" + coordinates + "</coordinates></cartographics></subject>")
    mods.write("<classification/>")
    mods.write("<name/>")
    mods.write("</mods>")
    mods.close()
    
    current_volume += 1

Now working on 0001
Now working on 0002
Now working on 0003
Now working on 0004
Now working on 0005
Now working on 0006
Now working on 0007
Now working on 0008
Now working on 0009
Now working on 0010
Now working on 0011
Now working on 0012
Now working on 0013
Now working on 0014
Now working on 0015
Now working on 0016
Now working on 0017
Now working on 0018
Now working on 0019
Now working on 0020
Now working on 0021
Now working on 0022
Now working on 0023
Now working on 0024
Now working on 0025
Now working on 0026
Now working on 0027
Now working on 0028
Now working on 0029
Now working on 0030
Now working on 0031
Now working on 0032
Now working on 0033
Now working on 0034
Now working on 0035
Now working on 0036
Now working on 0037
Now working on 0038
Now working on 0039
Now working on 0040
Now working on 0041
Now working on 0042
Now working on 0043
Now working on 0044
Now working on 0045
Now working on 0046
Now working on 0047
Now working on 0048
Now working on 0049
Now working on 0050


In [25]:
print(formats[6])

1 handwritten volume with 411 folios including the cover and inside cover 


In [14]:
print(4>3<2)

False


In [18]:
one = "I am nice."
two = "She is nicer."
print(one)
print(two)
print(one + two)
print('\n')
print(one + '\n' + '\n' + two)
#print(one + '\r' + two)

I am nice.
She is nicer.
I am nice.She is nicer.


I am nice.

She is nicer.


In [21]:
word = 'dog'
print('\\')

\


In [30]:
#IHGP metadata generation

master_xml_directory = input('Enter absolute directory of xml master: ')
master_xml = open(master_xml_directory,"r",encoding='utf-8')

titles = []
abstracts = []
dates = []
formats = []
conditions = []
languages = []
sources = []
folders = []
files = []
extents = []
creators = []

project = "British Library EAP627"
publisher = "Slave Societies Digital Archive"
director = "Courtney J. Campbell(project director)"
metadata ="Daniel Genkins(metadata creation)"
rights = "Vanderbilt University makes these historical records available for the limited purposes of individual private study, scholarship, research, teaching, and other educational or instructional uses. No rights of any kind are granted to users with respect to photographs or graphic images published on this site."
country = "Brazil"
continent = "South America"
state = "Paraíba"
city = "João Pessoa"
coordinates = "-7.11799,-34.88472"
types = ["Text","Historical records"]
eras = ["Early Modern","Modern"]
subjects = ["Writs--Wills--Deeds","Royal Orders","Governmental Correspondence","Notarial Records","Justice","Slave Emancipation","Revenue--Transportation","Provincial Legislation--State Constitution","Political Party","Geographic Atlas","Prayer Books","Colonial Documents--Imperial Documents--Republican Documents"]

for line in master_xml:
    line_content = line[line.find(">") + 1: line.rfind("<")]   
    
    if "<title>" in line:
        titles.append(line_content)
    elif "<description>" in line:
        abstracts.append(line_content)
    elif "<date>" in line:
        dates.append(line_content)
    elif "<date/>" in line:
        dates.append('-1')
    elif "<format>" in line:
        formats.append(line_content)
    elif "<creator>" in line:
        creators.append(line_content)
    elif "<condition>" in line:
        conditions.append(line_content)
    elif "<condition/>" in line:
        conditions.append('')
    elif "<language>" in line:
        languages.append(line_content)
    elif "<source>" in line:
        sources.append(line_content)
    elif "<folder>" in line:
        folders.append(line_content)
    elif "<file>" in line:
        files.append(line_content)
    elif "<extent>" in line:
        extents.append(line_content)
        
current_volume = 0

while current_volume <= 81:
    current_folder = "0" * (4 - len(str(current_volume + 1))) + str(current_volume + 1)
    print("Now working on " + current_folder)
    volume_directory = "\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\EAP627_ingest\\IHGP\\" + current_folder
    current_dc = open(volume_directory + "\\DC.xml","w",encoding = "utf-8")
    current_dc.write("<oai_dc:dc xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:oai_dc=\"http://www.openarchives.org/OAI/2.0/oai_dc/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd\">")
    current_dc.write("<title>" + titles[current_volume] + "</title>")
    current_dc.write("<creator>" + project + "</creator>")
    current_dc.write("<creator>" + creators[current_volume] + "</creator>")
    has_date = True
    if (0 <= current_volume <= 27) or (43 <= current_volume <= 47) or (current_volume == 80) or (current_volume == 33) or (68 <= current_volume <= 73):
        era = eras[0]        
    elif (current_volume == 66) or (current_volume == 81):
        has_date = False
    else:
        era = eras[1]
    if (current_volume == 0):
        subject = subjects[0]
    elif (1 <= current_volume <= 7):
        subject = subjects[1]
    elif (8 <= current_volume <= 13):
        subject = subjects[2]
    elif (14 <= current_volume <= 32):
        subject = subjects[3]
    elif (33 <= current_volume <= 37):
        subject = subjects[4]
    elif (38 <= current_volume <= 39):
        subject = subjects[5]
    elif (40 <= current_volume <= 42):
        subject = subjects[6]
    elif (43 <= current_volume <= 60):
        subject = subjects[7]
    elif (61 <= current_volume <= 63):
        subject = subjects[8]
    elif (64 <= current_volume <= 65):
        subject = subjects[9]
    elif (66 <= current_volume <= 67):
        subject = subjects[10]
    elif (68 <= current_volume <= 81):
        subject = subjects[11]
    if has_date:
        current_dc.write("<subject>" + subject + '--' + country + '--' + era + "</subject>")
    else:
        current_dc.write("<subject>" + subject + '--' + country + "</subject>")
    description = abstracts[current_volume] + " " + conditions[current_volume]
    current_dc.write("<description>" + description + "</description>")
    current_dc.write("<publisher>" + publisher + "</publisher>")
    current_dc.write("<contributor>" + director + "</contributor>")
    current_dc.write("<contributor>" + metadata + "</contributor>")
    current_dc.write("<source>" + sources[current_volume] + "</source>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
        current_dc.write("<date>" + date + "</date>")
    for typ in types:
        current_dc.write("<type>" + typ + "</type>")
    current_dc.write("<format>" + formats[current_volume] + "</format>")
    current_dc.write("<format>" + extents[current_volume] + "</format>")
    if (languages[current_volume] == "Portuguese"):
        current_dc.write("<language>por</language>")
    elif (languages[current_volume] == "Latin"):
        current_dc.write("<language>lat</language>")
    else:
        print("Bad language: " + str(current_volume))
    current_dc.write("<coverage>" + country + "</coverage>")
    current_dc.write("<coverage>" + continent + "--" + country + "--" + state + "--" + city + "</coverage>")
    current_dc.write("<coverage>" + coordinates + "</coverage>")
    current_dc.write("<coverage>" + era + "</coverage>")
    current_dc.write("<rights>" + rights + "</rights>")        
    current_dc.write("</oai_dc:dc>")
    current_dc.close()
    print("DC completed for " + current_folder)
    
    mods = open(volume_directory + "\\MODS.xml","w",encoding = "utf-8")
    mods.write("<mods xmlns=\"http://www.loc.gov/mods/v3\" xmlns:mods=\"http://www.loc.gov/mods/v3\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">")
    mods.write("<titleInfo>")
    mods.write("<title>" + titles[current_volume] + "</title>")
    mods.write("<subTitle/></titleInfo>")
    mods.write("<name type=\"personal\"><namePart>Courtney J. Campbell</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Project Director</roleTerm></role></name>")
    mods.write("<name type=\"personal\"><namePart>Daniel Genkins</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Metadata Creation</roleTerm></role></name>")
    mods.write("<typeOfResource>text</typeOfResource>")
    mods.write("<genre authority=\"marcgt\" />")
    mods.write("<tableOfContents/>")
    mods.write("<originInfo><dateIssued></dateIssued><copyrightDate/><issuance>monographic</issuance><edition/><publisher/><place/></originInfo>")
    if (languages[current_volume] == "Portuguese"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">por</languageTerm></language>")
    elif (languages[current_volume] == "Latin"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">lat</languageTerm></language>")
    mods.write("<abstract>" + abstracts[current_volume] + "</abstract>")
    mods.write("<identifier type=\"isbn\"/>")
    mods.write("<physicalDescription><form>" + conditions[current_volume] + "</form><extent>" + extents[current_volume]+ "</extent></physicalDescription>")
    mods.write("<note/>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
    mods.write("<subject><topic/><geographic>" + sources[current_volume] + "," + city + "," + state + "," + country + "</geographic><temporal>" + date + "</temporal>")
    mods.write("<hierarchicalGeographic><continent>" + continent + "</continent><country>" + country + "</country><province>" + state + "</province><region/><county/><city>" + city + "</city><citySection/></hierarchicalGeographic>")
    mods.write("<cartographics><coordinates>" + coordinates + "</coordinates></cartographics></subject>")
    mods.write("<classification/>")
    mods.write("<name/>")
    mods.write("</mods>")
    mods.close()
    print("MODS completed for " + current_folder)
    
    current_volume += 1 

print("All metadata completed.")

Enter absolute directory of xml master: C:\Users\genkindn\BL Metadata\EAP627\627_ihgp_master.xml
Now working on 0001
DC completed for 0001
MODS completed for 0001
Now working on 0002
DC completed for 0002
MODS completed for 0002
Now working on 0003
DC completed for 0003
MODS completed for 0003
Now working on 0004
DC completed for 0004
MODS completed for 0004
Now working on 0005
DC completed for 0005
MODS completed for 0005
Now working on 0006
DC completed for 0006
MODS completed for 0006
Now working on 0007
DC completed for 0007
MODS completed for 0007
Now working on 0008
DC completed for 0008
MODS completed for 0008
Now working on 0009
DC completed for 0009
MODS completed for 0009
Now working on 0010
DC completed for 0010
MODS completed for 0010
Now working on 0011
DC completed for 0011
MODS completed for 0011
Now working on 0012
DC completed for 0012
MODS completed for 0012
Now working on 0013
DC completed for 0013
MODS completed for 0013
Now working on 0014
DC completed for 0014
MOD

In [32]:
#AHP metadata generation

master_xml_directory = input('Enter absolute directory of xml master: ')
master_xml = open(master_xml_directory,"r",encoding='utf-8')

titles = []
abstracts = []
dates = []
formats = []
conditions = []
languages = []
sources = []
folders = []
files = []
extents = []
creators = []

project = "British Library EAP627"
publisher = "Slave Societies Digital Archive"
director = "Courtney J. Campbell(project director)"
metadata ="Daniel Genkins(metadata creation)"
rights = "Vanderbilt University makes these historical records available for the limited purposes of individual private study, scholarship, research, teaching, and other educational or instructional uses. No rights of any kind are granted to users with respect to photographs or graphic images published on this site."
country = "Brazil"
continent = "South America"
state = "Paraíba"
city = "João Pessoa"
coordinates = "-7.12174,-34.84525"
types = ["Text","Historical records"]
eras = ["Early Modern","Modern"]
subjects = ["Land Grants","Land Titles","Administrative Acts","Rank Letters","Paraíba Government","Mayoral Authority--Police Authority","Public Education","Municipal Council--Provincial Assembly","Provincial Treasury","Revenue Administration","Magistrates"]

for line in master_xml:
    line_content = line[line.find(">") + 1: line.rfind("<")]   
    
    if "<title>" in line:
        titles.append(line_content)
    elif "<description>" in line:
        abstracts.append(line_content)
    elif "<date>" in line:
        dates.append(line_content)
    elif "<date/>" in line:
        dates.append('-1')
    elif "<format>" in line:
        formats.append(line_content)
    elif "<creator>" in line:
        creators.append(line_content)
    elif "<creator/>" in line:
        creators.append('null')
    elif "<condition>" in line:
        conditions.append(line_content)
    elif "<condition/>" in line:
        conditions.append('')
    elif "<language>" in line:
        languages.append(line_content)
    elif "<source>" in line:
        sources.append(line_content)
    elif "<folder>" in line:
        folders.append(line_content)
    elif "<file>" in line:
        files.append(line_content)
    elif "<extent>" in line:
        extents.append(line_content)
        
current_volume = 0

while current_volume <= 127:
    current_folder = "0" * (4 - len(str(current_volume + 1))) + str(current_volume + 1)
    print("Now working on " + current_folder)
    volume_directory = "\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\EAP627_ingest\\AHP\\" + current_folder
    current_dc = open(volume_directory + "\\DC.xml","w",encoding = "utf-8")
    current_dc.write("<oai_dc:dc xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:oai_dc=\"http://www.openarchives.org/OAI/2.0/oai_dc/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd\">")
    current_dc.write("<title>" + titles[current_volume] + "</title>")
    current_dc.write("<creator>" + project + "</creator>")
    if creators[current_volume] != 'null':
        current_dc.write("<creator>" + creators[current_volume] + "</creator>")
    if (0 <= current_volume <= 10) or (20 <= current_volume <= 24) or (current_volume == 26) or (83 <= current_volume <= 86) or (28 <= current_volume <= 42) or (105 <= current_volume <= 107) or (119 <= current_volume <= 122):
        era = eras[0]        
    else:
        era = eras[1]
    if (0 <= current_volume <= 10):
        subject = subjects[0]
    elif (11 <= current_volume <= 19):
        subject = subjects[1]
    elif (20 <= current_volume <= 22):
        subject = subjects[2]
    elif (23 <= current_volume <= 27):
        subject = subjects[3]
    elif (28 <= current_volume <= 82):
        subject = subjects[4]
    elif (83 <= current_volume <= 99):
        subject = subjects[5]
    elif (100 <= current_volume <= 104):
        subject = subjects[6]
    elif (105 <= current_volume <= 106):
        subject = subjects[7]
    elif (107 <= current_volume <= 118):
        subject = subjects[8]
    elif (119 <= current_volume <= 124):
        subject = subjects[9]
    elif (125 <= current_volume <= 127):
        subject = subjects[10]
    current_dc.write("<subject>" + subject + '--' + country + '--' + era + "</subject>")
    description = abstracts[current_volume] + " " + conditions[current_volume]
    current_dc.write("<description>" + description + "</description>")
    current_dc.write("<publisher>" + publisher + "</publisher>")
    current_dc.write("<contributor>" + director + "</contributor>")
    current_dc.write("<contributor>" + metadata + "</contributor>")
    current_dc.write("<source>" + sources[current_volume] + "</source>")
    if (dates[current_volume].find('-') == -1):
        date = dates[current_volume]
    else:
        start_date = dates[current_volume][0:4]
        end_date = dates[current_volume][5:9]
        date = start_date + '/' + end_date
    current_dc.write("<date>" + date + "</date>")
    for typ in types:
        current_dc.write("<type>" + typ + "</type>")
    current_dc.write("<format>" + formats[current_volume] + "</format>")
    current_dc.write("<format>" + extents[current_volume] + "</format>")
    if (languages[current_volume] == "Portuguese"):
        current_dc.write("<language>por</language>")
    elif (languages[current_volume] == "Latin"):
        current_dc.write("<language>lat</language>")
    else:
        print("Bad language: " + str(current_volume))
    current_dc.write("<coverage>" + country + "</coverage>")
    current_dc.write("<coverage>" + continent + "--" + country + "--" + state + "--" + city + "</coverage>")
    current_dc.write("<coverage>" + coordinates + "</coverage>")
    current_dc.write("<coverage>" + era + "</coverage>")
    current_dc.write("<rights>" + rights + "</rights>")        
    current_dc.write("</oai_dc:dc>")
    current_dc.close()
    print("DC completed for " + current_folder)
    
    mods = open(volume_directory + "\\MODS.xml","w",encoding = "utf-8")
    mods.write("<mods xmlns=\"http://www.loc.gov/mods/v3\" xmlns:mods=\"http://www.loc.gov/mods/v3\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">")
    mods.write("<titleInfo>")
    mods.write("<title>" + titles[current_volume] + "</title>")
    mods.write("<subTitle/></titleInfo>")
    mods.write("<name type=\"personal\"><namePart>Courtney J. Campbell</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Project Director</roleTerm></role></name>")
    mods.write("<name type=\"personal\"><namePart>Daniel Genkins</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Metadata Creation</roleTerm></role></name>")
    mods.write("<typeOfResource>text</typeOfResource>")
    mods.write("<genre authority=\"marcgt\" />")
    mods.write("<tableOfContents/>")
    mods.write("<originInfo><dateIssued></dateIssued><copyrightDate/><issuance>monographic</issuance><edition/><publisher/><place/></originInfo>")
    if (languages[current_volume] == "Portuguese"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">por</languageTerm></language>")
    elif (languages[current_volume] == "Latin"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">lat</languageTerm></language>")
    mods.write("<abstract>" + abstracts[current_volume] + "</abstract>")
    mods.write("<identifier type=\"isbn\"/>")
    mods.write("<physicalDescription><form>" + conditions[current_volume] + "</form><extent>" + extents[current_volume]+ "</extent></physicalDescription>")
    mods.write("<note/>")
    mods.write("<subject><topic/><geographic>" + sources[current_volume] + "," + city + "," + state + "," + country + "</geographic><temporal>" + date + "</temporal>")
    mods.write("<hierarchicalGeographic><continent>" + continent + "</continent><country>" + country + "</country><province>" + state + "</province><region/><county/><city>" + city + "</city><citySection/></hierarchicalGeographic>")
    mods.write("<cartographics><coordinates>" + coordinates + "</coordinates></cartographics></subject>")
    mods.write("<classification/>")
    mods.write("<name/>")
    mods.write("</mods>")
    mods.close()
    print("MODS completed for " + current_folder)
    
    current_volume += 1 

print("All metadata completed.")

Enter absolute directory of xml master: C:\Users\genkindn\BL Metadata\EAP627\627_ahp_master.xml
Now working on 0001
DC completed for 0001
MODS completed for 0001
Now working on 0002
DC completed for 0002
MODS completed for 0002
Now working on 0003
DC completed for 0003
MODS completed for 0003
Now working on 0004
DC completed for 0004
MODS completed for 0004
Now working on 0005
DC completed for 0005
MODS completed for 0005
Now working on 0006
DC completed for 0006
MODS completed for 0006
Now working on 0007
DC completed for 0007
MODS completed for 0007
Now working on 0008
DC completed for 0008
MODS completed for 0008
Now working on 0009
DC completed for 0009
MODS completed for 0009
Now working on 0010
DC completed for 0010
MODS completed for 0010
Now working on 0011
DC completed for 0011
MODS completed for 0011
Now working on 0012
DC completed for 0012
MODS completed for 0012
Now working on 0013
DC completed for 0013
MODS completed for 0013
Now working on 0014
DC completed for 0014
MODS

In [1]:
#MATRIX NSM metadata generation

master_xml_directory = input('Enter absolute directory of xml master: ')
master_xml = open(master_xml_directory,"r",encoding='utf-8')

titles = []
abstracts = []
dates = []
formats = []
conditions = []
languages = []
sources = []
folders = []
files = []
extents = []

project = "British Library EAP627"
publisher = "Slave Societies Digital Archive"
director = "Courtney J. Campbell(project director)"
metadata ="Daniel Genkins(metadata creation)"
rights = "Vanderbilt University makes these historical records available for the limited purposes of individual private study, scholarship, research, teaching, and other educational or instructional uses. No rights of any kind are granted to users with respect to photographs or graphic images published on this site."
country = "Brazil"
continent = "South America"
state = "Paraíba"
city = "São João do Cariri"
coordinates = "-7.39077,-36.53513"
types = ["Text","Ecclesiastical records"]
eras = ["Early Modern","Modern"]
subjects = ["Baptisms--Marriages--Deaths","Baptisms","Confirmations","Marriages","Deaths","Finances","Kinship"]

for line in master_xml:
    line_content = line[line.find(">") + 1: line.rfind("<")]   
    
    if "<title>" in line:
        titles.append(line_content)
    elif "<description>" in line:
        abstracts.append(line_content)
    elif "<date>" in line:
        dates.append(line_content)
    elif "<date/>" in line:
        dates.append('-1')
    elif "<format>" in line:
        formats.append(line_content)
    elif "<condition>" in line:
        conditions.append(line_content)
    elif "<language>" in line:
        languages.append(line_content)
    elif "<source>" in line:
        sources.append(line_content)
    elif "<folder>" in line:
        folders.append(line_content)
    elif "<file>" in line:
        files.append(line_content)
    elif "<extent>" in line:
        extents.append(line_content)
        
current_volume = 0

while current_volume <= 53:
    current_folder = "0" * (4 - len(str(current_volume + 1))) + str(current_volume + 1)
    print("Now working on " + current_folder)
    volume_directory = "\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\SSDA_volume_metadata\\EAP627\\Paróquia_de_Nossa_Senhora_dos_Milagres"
    current_dc = open(volume_directory + "\\" + current_folder + "DC.xml","w",encoding = "utf-8")
    current_dc.write("<oai_dc:dc xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:oai_dc=\"http://www.openarchives.org/OAI/2.0/oai_dc/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd\">")
    current_dc.write("<title>" + titles[current_volume] + "</title>")
    current_dc.write("<creator>" + project + "</creator>")
    has_date = True
    if (0 <= current_volume <= 7) or (30 <= current_volume <= 32) or (current_volume == 39) or (45 <= current_volume <= 47):
        era = eras[0]        
    elif (50 <= current_volume <= 53):
        has_date = False
    else:
        era = eras[1]
    if (current_volume == 0):
        subject = subjects[0]
    elif (1 <= current_volume <= 29):
        subject = subjects[1]
    elif (current_volume == 30):
        subject = subjects[2]
    elif (31 <= current_volume <= 38):
        subject = subjects[3]
    elif (39 <= current_volume <= 44):
        subject = subjects[4]
    elif (45 <= current_volume <= 48):
        subject = subjects[5]
    elif (49 <= current_volume <= 53):
        subject = subjects[6]
    if has_date:
        current_dc.write("<subject>" + subject + '--' + country + '--' + era + "</subject>")
    else:
        current_dc.write("<subject>" + subject + '--' + country + "</subject>")
    description = abstracts[current_volume] + " " + conditions[current_volume]
    current_dc.write("<description>" + description + "</description>")
    current_dc.write("<publisher>" + publisher + "</publisher>")
    current_dc.write("<contributor>" + director + "</contributor>")
    current_dc.write("<contributor>" + metadata + "</contributor>")
    current_dc.write("<source>" + sources[current_volume] + "</source>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
        current_dc.write("<date>" + date + "</date>")
    for typ in types:
        current_dc.write("<type>" + typ + "</type>")
    current_dc.write("<format>" + formats[current_volume] + "</format>")
    current_dc.write("<format>" + extents[current_volume] + "</format>")
    if (languages[current_volume] == "Portuguese"):
        current_dc.write("<language>por</language>")
    else:
        print("Bad language: " + str(current_volume))
    current_dc.write("<coverage>" + country + "</coverage>")
    current_dc.write("<coverage>" + continent + "--" + country + "--" + state + "--" + city + "</coverage>")
    current_dc.write("<coverage>" + coordinates + "</coverage>")
    current_dc.write("<coverage>" + era + "</coverage>")
    current_dc.write("<rights>" + rights + "</rights>")        
    current_dc.write("</oai_dc:dc>")
    current_dc.close()
    
    mods = open(volume_directory + "\\" + current_folder + "MODS.xml","w",encoding = "utf-8")
    mods.write("<mods xmlns=\"http://www.loc.gov/mods/v3\" xmlns:mods=\"http://www.loc.gov/mods/v3\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">")
    mods.write("<titleInfo>")
    mods.write("<title>" + titles[current_volume] + "</title>")
    mods.write("<subTitle/></titleInfo>")
    mods.write("<name type=\"personal\"><namePart>Courtney J. Campbell</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Project Director</roleTerm></role></name>")
    mods.write("<name type=\"personal\"><namePart>Daniel Genkins</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Metadata Creation</roleTerm></role></name>")
    mods.write("<typeOfResource>text</typeOfResource>")
    mods.write("<genre authority=\"marcgt\" />")
    mods.write("<tableOfContents/>")
    mods.write("<originInfo><dateIssued></dateIssued><copyrightDate/><issuance>monographic</issuance><edition/><publisher/><place/></originInfo>")
    mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">por</languageTerm></language>")
    mods.write("<abstract>" + abstracts[current_volume] + "</abstract>")
    mods.write("<identifier type=\"isbn\"/>")
    mods.write("<physicalDescription><form>" + conditions[current_volume] + "</form><extent>" + extents[current_volume]+ "</extent></physicalDescription>")
    mods.write("<note/>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
    mods.write("<subject><topic/><geographic>" + sources[current_volume] + "," + city + "," + state + "," + country + "</geographic><temporal>" + date + "</temporal>")
    mods.write("<hierarchicalGeographic><continent>" + continent + "</continent><country>" + country + "</country><province>" + state + "</province><region/><county/><city>" + city + "</city><citySection/></hierarchicalGeographic>")
    mods.write("<cartographics><coordinates>" + coordinates + "</coordinates></cartographics></subject>")
    mods.write("<classification/>")
    mods.write("<name/>")
    mods.write("</mods>")
    mods.close()    
    
    current_volume += 1

Enter absolute directory of xml master: \\libshare.library.vanderbilt.edu\vulibrary\Landers\new_organization\Excel_metadata\EAP_627\627_nsm_master.xml
Now working on 0001
Now working on 0002
Now working on 0003
Now working on 0004
Now working on 0005
Now working on 0006
Now working on 0007
Now working on 0008
Now working on 0009
Now working on 0010
Now working on 0011
Now working on 0012
Now working on 0013
Now working on 0014
Now working on 0015
Now working on 0016
Now working on 0017
Now working on 0018
Now working on 0019
Now working on 0020
Now working on 0021
Now working on 0022
Now working on 0023
Now working on 0024
Now working on 0025
Now working on 0026
Now working on 0027
Now working on 0028
Now working on 0029
Now working on 0030
Now working on 0031
Now working on 0032
Now working on 0033
Now working on 0034
Now working on 0035
Now working on 0036
Now working on 0037
Now working on 0038
Now working on 0039
Now working on 0040
Now working on 0041
Now working on 0042
Now worki

In [2]:
#MATRIX IHGP metadata generation

master_xml_directory = input('Enter absolute directory of xml master: ')
master_xml = open(master_xml_directory,"r",encoding='utf-8')

titles = []
abstracts = []
dates = []
formats = []
conditions = []
languages = []
sources = []
folders = []
files = []
extents = []
creators = []

project = "British Library EAP627"
publisher = "Slave Societies Digital Archive"
director = "Courtney J. Campbell(project director)"
metadata ="Daniel Genkins(metadata creation)"
rights = "Vanderbilt University makes these historical records available for the limited purposes of individual private study, scholarship, research, teaching, and other educational or instructional uses. No rights of any kind are granted to users with respect to photographs or graphic images published on this site."
country = "Brazil"
continent = "South America"
state = "Paraíba"
city = "João Pessoa"
coordinates = "-7.11799,-34.88472"
types = ["Text","Historical records"]
eras = ["Early Modern","Modern"]
subjects = ["Writs--Wills--Deeds","Royal Orders","Governmental Correspondence","Notarial Records","Justice","Slave Emancipation","Revenue--Transportation","Provincial Legislation--State Constitution","Political Party","Geographic Atlas","Prayer Books","Colonial Documents--Imperial Documents--Republican Documents"]

for line in master_xml:
    line_content = line[line.find(">") + 1: line.rfind("<")]   
    
    if "<title>" in line:
        titles.append(line_content)
    elif "<description>" in line:
        abstracts.append(line_content)
    elif "<date>" in line:
        dates.append(line_content)
    elif "<date/>" in line:
        dates.append('-1')
    elif "<format>" in line:
        formats.append(line_content)
    elif "<creator>" in line:
        creators.append(line_content)
    elif "<condition>" in line:
        conditions.append(line_content)
    elif "<condition/>" in line:
        conditions.append('')
    elif "<language>" in line:
        languages.append(line_content)
    elif "<source>" in line:
        sources.append(line_content)
    elif "<folder>" in line:
        folders.append(line_content)
    elif "<file>" in line:
        files.append(line_content)
    elif "<extent>" in line:
        extents.append(line_content)
        
current_volume = 0

while current_volume <= 81:
    current_folder = "0" * (4 - len(str(current_volume + 1))) + str(current_volume + 1)
    print("Now working on " + current_folder)
    volume_directory = "\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\SSDA_volume_metadata\\EAP627\\Instituto_Histórico_e_Geográfico_Paraibano"
    current_dc = open(volume_directory + "\\" + current_folder + "DC.xml","w",encoding = "utf-8")
    current_dc.write("<oai_dc:dc xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:oai_dc=\"http://www.openarchives.org/OAI/2.0/oai_dc/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd\">")
    current_dc.write("<title>" + titles[current_volume] + "</title>")
    current_dc.write("<creator>" + project + "</creator>")
    current_dc.write("<creator>" + creators[current_volume] + "</creator>")
    has_date = True
    if (0 <= current_volume <= 27) or (43 <= current_volume <= 47) or (current_volume == 80) or (current_volume == 33) or (68 <= current_volume <= 73):
        era = eras[0]        
    elif (current_volume == 66) or (current_volume == 81):
        has_date = False
    else:
        era = eras[1]
    if (current_volume == 0):
        subject = subjects[0]
    elif (1 <= current_volume <= 7):
        subject = subjects[1]
    elif (8 <= current_volume <= 13):
        subject = subjects[2]
    elif (14 <= current_volume <= 32):
        subject = subjects[3]
    elif (33 <= current_volume <= 37):
        subject = subjects[4]
    elif (38 <= current_volume <= 39):
        subject = subjects[5]
    elif (40 <= current_volume <= 42):
        subject = subjects[6]
    elif (43 <= current_volume <= 60):
        subject = subjects[7]
    elif (61 <= current_volume <= 63):
        subject = subjects[8]
    elif (64 <= current_volume <= 65):
        subject = subjects[9]
    elif (66 <= current_volume <= 67):
        subject = subjects[10]
    elif (68 <= current_volume <= 81):
        subject = subjects[11]
    if has_date:
        current_dc.write("<subject>" + subject + '--' + country + '--' + era + "</subject>")
    else:
        current_dc.write("<subject>" + subject + '--' + country + "</subject>")
    description = abstracts[current_volume] + " " + conditions[current_volume]
    current_dc.write("<description>" + description + "</description>")
    current_dc.write("<publisher>" + publisher + "</publisher>")
    current_dc.write("<contributor>" + director + "</contributor>")
    current_dc.write("<contributor>" + metadata + "</contributor>")
    current_dc.write("<source>" + sources[current_volume] + "</source>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
        current_dc.write("<date>" + date + "</date>")
    for typ in types:
        current_dc.write("<type>" + typ + "</type>")
    current_dc.write("<format>" + formats[current_volume] + "</format>")
    current_dc.write("<format>" + extents[current_volume] + "</format>")
    if (languages[current_volume] == "Portuguese"):
        current_dc.write("<language>por</language>")
    elif (languages[current_volume] == "Latin"):
        current_dc.write("<language>lat</language>")
    else:
        print("Bad language: " + str(current_volume))
    current_dc.write("<coverage>" + country + "</coverage>")
    current_dc.write("<coverage>" + continent + "--" + country + "--" + state + "--" + city + "</coverage>")
    current_dc.write("<coverage>" + coordinates + "</coverage>")
    current_dc.write("<coverage>" + era + "</coverage>")
    current_dc.write("<rights>" + rights + "</rights>")        
    current_dc.write("</oai_dc:dc>")
    current_dc.close()
    print("DC completed for " + current_folder)
    
    mods = open(volume_directory + "\\" + current_folder + "MODS.xml","w",encoding = "utf-8")
    mods.write("<mods xmlns=\"http://www.loc.gov/mods/v3\" xmlns:mods=\"http://www.loc.gov/mods/v3\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">")
    mods.write("<titleInfo>")
    mods.write("<title>" + titles[current_volume] + "</title>")
    mods.write("<subTitle/></titleInfo>")
    mods.write("<name type=\"personal\"><namePart>Courtney J. Campbell</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Project Director</roleTerm></role></name>")
    mods.write("<name type=\"personal\"><namePart>Daniel Genkins</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Metadata Creation</roleTerm></role></name>")
    mods.write("<typeOfResource>text</typeOfResource>")
    mods.write("<genre authority=\"marcgt\" />")
    mods.write("<tableOfContents/>")
    mods.write("<originInfo><dateIssued></dateIssued><copyrightDate/><issuance>monographic</issuance><edition/><publisher/><place/></originInfo>")
    if (languages[current_volume] == "Portuguese"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">por</languageTerm></language>")
    elif (languages[current_volume] == "Latin"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">lat</languageTerm></language>")
    mods.write("<abstract>" + abstracts[current_volume] + "</abstract>")
    mods.write("<identifier type=\"isbn\"/>")
    mods.write("<physicalDescription><form>" + conditions[current_volume] + "</form><extent>" + extents[current_volume]+ "</extent></physicalDescription>")
    mods.write("<note/>")
    if (dates[current_volume] != '-1'):
        if (dates[current_volume].find('-') == -1):
            date = dates[current_volume]
        else:
            start_date = dates[current_volume][0:4]
            end_date = dates[current_volume][5:9]
            date = start_date + '/' + end_date
    mods.write("<subject><topic/><geographic>" + sources[current_volume] + "," + city + "," + state + "," + country + "</geographic><temporal>" + date + "</temporal>")
    mods.write("<hierarchicalGeographic><continent>" + continent + "</continent><country>" + country + "</country><province>" + state + "</province><region/><county/><city>" + city + "</city><citySection/></hierarchicalGeographic>")
    mods.write("<cartographics><coordinates>" + coordinates + "</coordinates></cartographics></subject>")
    mods.write("<classification/>")
    mods.write("<name/>")
    mods.write("</mods>")
    mods.close()
    print("MODS completed for " + current_folder)
    
    current_volume += 1 

print("All metadata completed.")

Enter absolute directory of xml master: \\libshare.library.vanderbilt.edu\vulibrary\Landers\new_organization\Excel_metadata\EAP_627\627_ihgp_master.xml
Now working on 0001
DC completed for 0001
MODS completed for 0001
Now working on 0002
DC completed for 0002
MODS completed for 0002
Now working on 0003
DC completed for 0003
MODS completed for 0003
Now working on 0004
DC completed for 0004
MODS completed for 0004
Now working on 0005
DC completed for 0005
MODS completed for 0005
Now working on 0006
DC completed for 0006
MODS completed for 0006
Now working on 0007
DC completed for 0007
MODS completed for 0007
Now working on 0008
DC completed for 0008
MODS completed for 0008
Now working on 0009
DC completed for 0009
MODS completed for 0009
Now working on 0010
DC completed for 0010
MODS completed for 0010
Now working on 0011
DC completed for 0011
MODS completed for 0011
Now working on 0012
DC completed for 0012
MODS completed for 0012
Now working on 0013
DC completed for 0013
MODS completed

In [3]:
#MATRIX AHP metadata generation

master_xml_directory = input('Enter absolute directory of xml master: ')
master_xml = open(master_xml_directory,"r",encoding='utf-8')

titles = []
abstracts = []
dates = []
formats = []
conditions = []
languages = []
sources = []
folders = []
files = []
extents = []
creators = []

project = "British Library EAP627"
publisher = "Slave Societies Digital Archive"
director = "Courtney J. Campbell(project director)"
metadata ="Daniel Genkins(metadata creation)"
rights = "Vanderbilt University makes these historical records available for the limited purposes of individual private study, scholarship, research, teaching, and other educational or instructional uses. No rights of any kind are granted to users with respect to photographs or graphic images published on this site."
country = "Brazil"
continent = "South America"
state = "Paraíba"
city = "João Pessoa"
coordinates = "-7.12174,-34.84525"
types = ["Text","Historical records"]
eras = ["Early Modern","Modern"]
subjects = ["Land Grants","Land Titles","Administrative Acts","Rank Letters","Paraíba Government","Mayoral Authority--Police Authority","Public Education","Municipal Council--Provincial Assembly","Provincial Treasury","Revenue Administration","Magistrates"]

for line in master_xml:
    line_content = line[line.find(">") + 1: line.rfind("<")]   
    
    if "<title>" in line:
        titles.append(line_content)
    elif "<description>" in line:
        abstracts.append(line_content)
    elif "<date>" in line:
        dates.append(line_content)
    elif "<date/>" in line:
        dates.append('-1')
    elif "<format>" in line:
        formats.append(line_content)
    elif "<creator>" in line:
        creators.append(line_content)
    elif "<creator/>" in line:
        creators.append('null')
    elif "<condition>" in line:
        conditions.append(line_content)
    elif "<condition/>" in line:
        conditions.append('')
    elif "<language>" in line:
        languages.append(line_content)
    elif "<source>" in line:
        sources.append(line_content)
    elif "<folder>" in line:
        folders.append(line_content)
    elif "<file>" in line:
        files.append(line_content)
    elif "<extent>" in line:
        extents.append(line_content)
        
current_volume = 0

while current_volume <= 127:
    current_folder = "0" * (4 - len(str(current_volume + 1))) + str(current_volume + 1)
    print("Now working on " + current_folder)
    volume_directory = "\\\\libshare.library.vanderbilt.edu\\vulibrary\\Landers\\new_organization\\SSDA_volume_metadata\\EAP627\\Arquivo_Histórico_da_Paraíba"
    current_dc = open(volume_directory + "\\" + current_folder + "DC.xml","w",encoding = "utf-8")
    current_dc.write("<oai_dc:dc xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:oai_dc=\"http://www.openarchives.org/OAI/2.0/oai_dc/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:schemaLocation=\"http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd\">")
    current_dc.write("<title>" + titles[current_volume] + "</title>")
    current_dc.write("<creator>" + project + "</creator>")
    if creators[current_volume] != 'null':
        current_dc.write("<creator>" + creators[current_volume] + "</creator>")
    if (0 <= current_volume <= 10) or (20 <= current_volume <= 24) or (current_volume == 26) or (83 <= current_volume <= 86) or (28 <= current_volume <= 42) or (105 <= current_volume <= 107) or (119 <= current_volume <= 122):
        era = eras[0]        
    else:
        era = eras[1]
    if (0 <= current_volume <= 10):
        subject = subjects[0]
    elif (11 <= current_volume <= 19):
        subject = subjects[1]
    elif (20 <= current_volume <= 22):
        subject = subjects[2]
    elif (23 <= current_volume <= 27):
        subject = subjects[3]
    elif (28 <= current_volume <= 82):
        subject = subjects[4]
    elif (83 <= current_volume <= 99):
        subject = subjects[5]
    elif (100 <= current_volume <= 104):
        subject = subjects[6]
    elif (105 <= current_volume <= 106):
        subject = subjects[7]
    elif (107 <= current_volume <= 118):
        subject = subjects[8]
    elif (119 <= current_volume <= 124):
        subject = subjects[9]
    elif (125 <= current_volume <= 127):
        subject = subjects[10]
    current_dc.write("<subject>" + subject + '--' + country + '--' + era + "</subject>")
    description = abstracts[current_volume] + " " + conditions[current_volume]
    current_dc.write("<description>" + description + "</description>")
    current_dc.write("<publisher>" + publisher + "</publisher>")
    current_dc.write("<contributor>" + director + "</contributor>")
    current_dc.write("<contributor>" + metadata + "</contributor>")
    current_dc.write("<source>" + sources[current_volume] + "</source>")
    if (dates[current_volume].find('-') == -1):
        date = dates[current_volume]
    else:
        start_date = dates[current_volume][0:4]
        end_date = dates[current_volume][5:9]
        date = start_date + '/' + end_date
    current_dc.write("<date>" + date + "</date>")
    for typ in types:
        current_dc.write("<type>" + typ + "</type>")
    current_dc.write("<format>" + formats[current_volume] + "</format>")
    current_dc.write("<format>" + extents[current_volume] + "</format>")
    if (languages[current_volume] == "Portuguese"):
        current_dc.write("<language>por</language>")
    elif (languages[current_volume] == "Latin"):
        current_dc.write("<language>lat</language>")
    else:
        print("Bad language: " + str(current_volume))
    current_dc.write("<coverage>" + country + "</coverage>")
    current_dc.write("<coverage>" + continent + "--" + country + "--" + state + "--" + city + "</coverage>")
    current_dc.write("<coverage>" + coordinates + "</coverage>")
    current_dc.write("<coverage>" + era + "</coverage>")
    current_dc.write("<rights>" + rights + "</rights>")        
    current_dc.write("</oai_dc:dc>")
    current_dc.close()
    print("DC completed for " + current_folder)
    
    mods = open(volume_directory + "\\" + current_folder + "MODS.xml","w",encoding = "utf-8")
    mods.write("<mods xmlns=\"http://www.loc.gov/mods/v3\" xmlns:mods=\"http://www.loc.gov/mods/v3\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">")
    mods.write("<titleInfo>")
    mods.write("<title>" + titles[current_volume] + "</title>")
    mods.write("<subTitle/></titleInfo>")
    mods.write("<name type=\"personal\"><namePart>Courtney J. Campbell</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Project Director</roleTerm></role></name>")
    mods.write("<name type=\"personal\"><namePart>Daniel Genkins</namePart><role><roleTerm authority=\"marcrelator\" type=\"text\">Metadata Creation</roleTerm></role></name>")
    mods.write("<typeOfResource>text</typeOfResource>")
    mods.write("<genre authority=\"marcgt\" />")
    mods.write("<tableOfContents/>")
    mods.write("<originInfo><dateIssued></dateIssued><copyrightDate/><issuance>monographic</issuance><edition/><publisher/><place/></originInfo>")
    if (languages[current_volume] == "Portuguese"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">por</languageTerm></language>")
    elif (languages[current_volume] == "Latin"):
        mods.write("<language><languageTerm authority=\"iso639-2b\" type=\"code\">lat</languageTerm></language>")
    mods.write("<abstract>" + abstracts[current_volume] + "</abstract>")
    mods.write("<identifier type=\"isbn\"/>")
    mods.write("<physicalDescription><form>" + conditions[current_volume] + "</form><extent>" + extents[current_volume]+ "</extent></physicalDescription>")
    mods.write("<note/>")
    mods.write("<subject><topic/><geographic>" + sources[current_volume] + "," + city + "," + state + "," + country + "</geographic><temporal>" + date + "</temporal>")
    mods.write("<hierarchicalGeographic><continent>" + continent + "</continent><country>" + country + "</country><province>" + state + "</province><region/><county/><city>" + city + "</city><citySection/></hierarchicalGeographic>")
    mods.write("<cartographics><coordinates>" + coordinates + "</coordinates></cartographics></subject>")
    mods.write("<classification/>")
    mods.write("<name/>")
    mods.write("</mods>")
    mods.close()
    print("MODS completed for " + current_folder)
    
    current_volume += 1 

print("All metadata completed.")

Enter absolute directory of xml master: \\libshare.library.vanderbilt.edu\vulibrary\Landers\new_organization\Excel_metadata\EAP_627\627_ahp_master.xml
Now working on 0001
DC completed for 0001
MODS completed for 0001
Now working on 0002
DC completed for 0002
MODS completed for 0002
Now working on 0003
DC completed for 0003
MODS completed for 0003
Now working on 0004
DC completed for 0004
MODS completed for 0004
Now working on 0005
DC completed for 0005
MODS completed for 0005
Now working on 0006
DC completed for 0006
MODS completed for 0006
Now working on 0007
DC completed for 0007
MODS completed for 0007
Now working on 0008
DC completed for 0008
MODS completed for 0008
Now working on 0009
DC completed for 0009
MODS completed for 0009
Now working on 0010
DC completed for 0010
MODS completed for 0010
Now working on 0011
DC completed for 0011
MODS completed for 0011
Now working on 0012
DC completed for 0012
MODS completed for 0012
Now working on 0013
DC completed for 0013
MODS completed 